In [195]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import pandas as pd
import numpy as np

# Import csv from preprocessing - generated with python script

In [196]:
test = pd.read_csv('../identification.csv')

In [197]:
test

,token,lemma,sentence,index,start,pos,sentiment,paragraph,docPosition,sentPosition,...,precedesLCA,followsLCAPath,precedesLCAPath,head,uppermost,uppermost_child,right_sibling,right_sibling_head,probability,IOB
0,It,it,0,1,55,PRP,Neutral,0,Introduction,First,...,NaN,0.166667,-1.000000,said-4,NP,PRP,PRP,said,0.500000,O
1,is,be,0,2,58,VBZ,Neutral,0,Introduction,Middle,...,S,0.250000,0.166667,said-4,VBZ,is,is,said,0.007090,O
2,always,always,0,3,61,RB,Neutral,0,Introduction,Middle,...,VP,0.250000,0.250000,said-4,ADVP,RB,RB,promote,0.007090,O
3,said,say,0,4,68,VBN,Neutral,0,Introduction,Middle,...,VP,0.333333,0.250000,said-4,ROOT,S,S,said,0.007090,O
4,that,that,0,5,73,IN,Neutral,0,Introduction,Middle,...,VP,0.416667,0.333333,promote-9,IN,that,that,promote,0.007090,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,in,in,0,34,2215,IN,Neutral,3,Conclusion,Middle,...,ADJP,0.750000,0.666667,success-37,IN,in,in,success,0.084201,Arg-I
404,one,one,0,35,2218,CD,Neutral,3,Conclusion,Middle,...,PP,0.916667,0.750000,success-37,NP,CD,POS,one,0.084201,Arg-I
405,'s,'s,0,36,2221,POS,Neutral,3,Conclusion,Middle,...,NP,0.833333,0.916667,one-35,POS,'s,'s,one,0.084201,Arg-I
406,success,success,0,37,2224,NN,Neutral,3,Conclusion,Middle,...,NP,0.166667,0.833333,profitable-33,PP,IN,NP,profitable,0.084201,Arg-I


## Preprocess data from csv/DataFrame

In [198]:
sent_x = []
sent_y = []
for p in set(test.paragraph.values):
    for s in set(test[test.paragraph==p].sentence.values):
        temp_test = test[(test.paragraph == p) & (test.sentence == s)]
        sent_x.append(temp_test.loc[:, temp_test.columns != 'IOB'].to_dict("record"))
        sent_y.append(temp_test.IOB.values)

/var/folders/7r/c0g604c13xn1wrqn69g540c00000gn/T/ipykernel_85540/1349379417.py:6: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  sent_x.append(temp_test.loc[:, temp_test.columns != 'IOB'].to_dict("record"))


## Split data - informal for now and only on esssay001

In [199]:
X_train = sent_x[:10]
y_train = sent_y[:10]
X_test = sent_x[10:]
y_test = sent_y[10:]

## Initialize CRF trainer

In [200]:
trainer = pycrfsuite.Trainer(verbose=False)


### add training data

In [201]:
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [202]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

## Train and save to file

In [203]:
trainer.train('conll2002-esp.crfsuite')

## Load model from file

In [204]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

## Test

In [205]:
print("Predicted:", ' '.join(tagger.tag(X_test[2])))
print("Correct:  ", ' '.join(y_test[2]))

Predicted: Arg-B Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I
Correct:   Arg-B Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I O


In [206]:
y_pred = []
for i, each in enumerate(X_test):
    y_pred.append(tagger.tag(each))


In [207]:
lb = LabelBinarizer()

In [208]:
y_true_combined = lb.fit_transform(list(chain.from_iterable(y_test)))

In [209]:
y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))


In [210]:
tagset = set(lb.classes_)
tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

report = classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
)

In [211]:
print(report)

              precision    recall  f1-score   support

       Arg-B       0.40      0.50      0.44         4
       Arg-I       0.79      0.88      0.83       102
           O       0.70      0.54      0.61        57

   micro avg       0.75      0.75      0.75       163
   macro avg       0.63      0.64      0.63       163
weighted avg       0.75      0.75      0.75       163
 samples avg       0.75      0.75      0.75       163

